In [1]:
# Subprocess allows us to issue commands on the command line
import subprocess
# Module to download files with an URL
import urllib.request
# Zip utlity to extract files
from zipfile import ZipFile

In [2]:
############### Project properties ###############

# Root GIS directory
myRoot = "d:\\MattsGIS"
myData = f"{myRoot}\\data"

# Data source URL RRG 
gdbURL ='https://astoria.nyc3.digitaloceanspaces.com/geo409/rrg_build.gdb.zip'

# Data source URL RKY
# gdbURL = 'https://outragegis.com/d/smallScaleKentucky.gdb.zip'

inputDatabase = "rrg_build.gdb"
outputDatabase = "rrg.gdb"

# Project name - created when you launched ArcGIS Pro
project = "rrg"

# Folder for downloaded assets. Will be wiped at end
downloads = f'{myRoot}\\data\\downloads'

arcpy.env.workspace = f"{myData}\\{inputDatabase}"
arcpy.env.overwriteOutput = True

In [3]:
# Create download folder
subprocess.run(f'mkdir {myData}', shell=True, stdout=subprocess.PIPE)
subprocess.run(f'mkdir {downloads}', shell=True, stdout=subprocess.PIPE)
print(f"mkdir {myData}\n")
    
# Show contents of project directory
completed = subprocess.run(f'dir {myRoot}\\{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

mkdir d:\MattsGIS\data

 Volume in drive D is WCDisk
 Volume Serial Number is EAED-D1D5

 Directory of d:\MattsGIS




In [4]:
# Download and extract assets
print(f'Downloading {gdbURL}')
with urllib.request.urlopen(gdbURL) as response: 
    with open(f'{downloads}\\{inputDatabase}.zip', 'wb') as outFile:
        data = response.read()
        outFile.write(data) 
with ZipFile(f'{downloads}\\{inputDatabase}.zip', 'r') as zip: 
    zip.extractall(f'{downloads}\\')

command = f'move {downloads}\\{inputDatabase} {myRoot}\\data\\'
print(command)
completed = subprocess.run(command, shell=True, stdout=subprocess.PIPE)

print(completed.stdout.decode('UTF-8'))
command = f'del {downloads}\\{inputDatabase}.zip'
completed = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

command = f'dir {myData}'
completed = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))


move d:\MattsGIS\data\downloads\rrg_build.gdb d:\MattsGIS\data\
        1 dir(s) moved.


 Volume in drive D is WCDisk
 Volume Serial Number is EAED-D1D5

 Directory of d:\MattsGIS\data

04/15/2020  11:40 AM    <DIR>          .
04/15/2020  11:40 AM    <DIR>          ..
04/15/2020  11:40 AM    <DIR>          downloads
04/15/2020  11:40 AM    <DIR>          rrg_build.gdb
               0 File(s)              0 bytes
               4 Dir(s)   5,290,696,704 bytes free



In [5]:
# get list of feature classes in our database
featureList = arcpy.ListFeatureClasses()
print (f"List of feature classes: {featureList}")

# get list of feature classes in our database
rasterList = arcpy.ListRasters()
print (f"List of rasters: {rasterList}")

List of feature classes: ['area_of_interest', 'ky_gnis_1808', 'streams_flowline', 'streams_water_areas', 'streams_waterbodies']
List of rasters: ['DEM_2016_5ft', 'Landcover_2012_30m']


In [6]:
for feature in featureList:
    # Create describe object
    d = arcpy.Describe(feature)
    print(f'{feature} is a {d.shapeType} with CRS: {d.spatialReference.name}')
    fields = arcpy.ListFields(feature)
    for field in fields:
        print(f"___{field.name} is a type of {field.type}")

area_of_interest is a Polygon with CRS: NAD83_Kentucky_Single_Zone_ftUS
___OBJECTID is a type of OID
___Shape is a type of Geometry
___FID_1 is a type of Double
___Shape_Length is a type of Double
___Shape_Area is a type of Double
ky_gnis_1808 is a Point with CRS: NAD_1983_StatePlane_Kentucky_FIPS_1600_Feet
___OBJECTID is a type of OID
___Shape is a type of Geometry
___FEATURE_ID is a type of Integer
___FEATURE_NAME is a type of String
___FEATURE_CLASS is a type of String
___STATE_ALPHA is a type of String
___STATE_NUMERIC is a type of Integer
___COUNTY_NAME is a type of String
___COUNTY_NUMERIC is a type of Integer
___PRIMARY_LAT_DMS is a type of String
___PRIM_LONG_DMS is a type of String
___PRIMARY_LAT_DMS_D is a type of Double
___PRIM_LONG_DMS_D is a type of Double
___PRIM_LAT_DEC is a type of Double
___PRIM_LONG_DEC is a type of Double
___SOURCE_LAT_DMS is a type of String
___SOURCE_LONG_DMS is a type of String
___SOURCE_LAT_DMS_D is a type of Double
___SOURCE_LONG_DMS_D is a type

In [7]:
rasterList = arcpy.ListRasters()
for raster in rasterList:
    print(arcpy.Exists(raster))
    d = arcpy.Describe(raster)
    linearUnit = d.SpatialReference.linearUnitName
#     minimum = arcpy.GetRasterProperties_management(raster, "MINIMUM")
#     maximum = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
#     mean = arcpy.GetRasterProperties_management(raster, "MEAN")
    
    print(f'{raster} is {d.width} x {d.height} pixels \n____with a {round(d.meanCellWidth, 1)} {linearUnit} resolution. Pixel type is {d.pixelType}.')
#     print(f'____min: {minimum} - mean: {mean} - max: {maximum}')

True
DEM_2016_5ft is 14620 x 11236 pixels 
____with a 5.0 Foot_US resolution. Pixel type is F32.
True
Landcover_2012_30m is 743 x 571 pixels 
____with a 98.4 Foot_US resolution. Pixel type is U8.


In [8]:
print("Creating workspace geodatabase...")
if arcpy.Exists(f"{myData}\\{outputDatabase}"):
    print(f"Output database exists: {myData}\\{outputDatabase}")
else:
    arcpy.CreateFileGDB_management(myData, outputDatabase)
    print(f"Created output database: {myData}\\{outputDatabase}")
    
print(f"Using {arcpy.env.workspace} for input data.")

Creating workspace geodatabase...
Created output database: d:\MattsGIS\data\rrg.gdb
Using d:\MattsGIS\data\rrg_build.gdb for input data.


In [9]:
clipLayer = "area_of_interest"

# Iterate over feature classes in input database
for feature in featureList:
    # Create output path for each feature class
    output = f"{myRoot}\\data\\{outputDatabase}\\{feature}"
    # Don't clip the AOI, just copy it
    if feature == clipLayer:
        arcpy.Copy_management(feature, output)
        print(f"Copying {feature} to {output}")
    else:
        print(f"Clipping {feature} with {clipLayer} and outputting layer to {output} ...")
        # Clip feature class to AOI
        arcpy.Clip_analysis(feature, clipLayer, output)

Copying area_of_interest to d:\MattsGIS\data\rrg.gdb\area_of_interest
Clipping ky_gnis_1808 with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\ky_gnis_1808 ...
Clipping streams_flowline with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\streams_flowline ...
Clipping streams_water_areas with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\streams_water_areas ...
Clipping streams_waterbodies with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\streams_waterbodies ...


In [10]:
for raster in rasterList:
    output = f"{myRoot}\\data\\{outputDatabase}\\{raster}"
    # arcpy.Clip_management(in_raster, {rectangle}, out_raster, {in_template_dataset}, {nodata_value}, {clipping_geometry}, {maintain_clipping_extent})
    print(f"Clipping {raster} with {clipLayer} and outputting layer to {output} ...")
    arcpy.Clip_management(raster, "#", output, clipLayer, "#", "ClippingGeometry","#")

Clipping DEM_2016_5ft with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\DEM_2016_5ft ...
Clipping Landcover_2012_30m with area_of_interest and outputting layer to d:\MattsGIS\data\rrg.gdb\Landcover_2012_30m ...


In [11]:
arcpy.env.workspace = f"{myRoot}\\data\\{outputDatabase}"

In [12]:
# Hillshade tool syntax
# arcpy.HillShade_3d(in_raster, out_raster, {azimuth}, {altitude}, {model_shadows}, {z_factor})
arcpy.HillShade_3d('DEM_2016_5ft', 'rrg_hillshade')

<Result 'd:\\MattsGIS\\data\\rrg.gdb\\rrg_hillshade'>